Testing openAI API

In [ ]:
from openai import OpenAI
# creating the model to be used
client = OpenAI(api_key="{{api_key}}")
# defining the model name and messages to display to check if the key works
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hi, my name is Alex."}
  ])
# printing the first message
print(completion.choices[0].message)


LLM model

In [ ]:
from langchain.llms import OpenAI
# Initiating the OpenAI LLM with API key
llm = OpenAI(api_key="{{api_key}}")
# Query the model
response = llm.invoke("What is the tallest building in the world?")
print(response)

Chat model

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage, SystemMessage
# Initiating the chat model with API key
chat = ChatOpenAI(api_key="{{api_key}}")
# Defines a context and query using SystemMessage and HumanMessage
messages = [
    SystemMessage(content="You are a math tutor who provides answers with a bit of sarcasm."),
    HumanMessage(content="What is the square of 2?"),
]
response = chat.invoke(messages)
print(response.content)


Basic prompt template

In [ ]:
# importing LangChain modules
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
# Initiating the chat model with API key
llm = OpenAI(openai_api_key = "{{api_key}}")
email_template = PromptTemplate.from_template(
    "Create an invitation email to the recipinet that is {recipient_name} \
 for an event that is {event_type} in a language that is {language} \
 Mention the event location that is {event_location} \
 and event date that is {event_date}. \
 Also write few sentences about the event description that is {event_description} \
 in style that is {style} "
)
message = email_template.format(
    style = "enthusiastic tone",
    language = "American english",
    recipient_name="John",
    event_type="product launch",
    event_date="January 15, 2024",
    event_location="Grand Ballroom, City Center Hotel",
    event_description="an exciting unveiling of our latest innovations"
    )
response = llm.invoke(message)
print(response)

Few-shot prompt

In [ ]:
# importing LangChain modules
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate
# Initiating the chat model with API key
chat = ChatOpenAI(temperature=0.0, openai_api_key = "{{api_key}}")
examples = [
  {
    "review": "I absolutely love this product! It exceeded my expectations.",
    "sentiment": "Positive"
  },
  {
    "review": "I'm really disappointed with the quality of this item. It didn't meet my needs.",
    "sentiment": "Negative"
  },
  {
    "review": "The product is okay, but there's room for improvement.",
    "sentiment": "Neutral"
  }
]
example_prompt = PromptTemplate(
                        input_variables=["review", "sentiment"],
                        template="Review: {review}\n{sentiment}")

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Review: {input}",
    input_variables=["input"]
)

message = prompt.format(input="The machine worked okay without much trouble.")

response = chat.invoke(message)
print(response.content)

Datetime parser

In [ ]:
from langchain.output_parsers import DatetimeOutputParser
parser_dateTime = DatetimeOutputParser()
# creating our prompt template
template = """Provide the response in format {format_instructions}
            to the user's question {question}"""

# passing the format instructions of the parser to the template
prompt_dateTime = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": parser_dateTime.get_format_instructions()},
)
print(llm.predict(text = prompt_dateTime.format(question="ENTER_YOUR_QUERY_HERE")))

List parser

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
parser_List = CommaSeparatedListOutputParser()
partial_variables={"format_instructions": parser_List.get_format_instructions()}
print(llm.predict(text = prompt_List.format(question="ENTER_YOUR_QUERY_HERE")))

Datetime and list parsers

In [ ]:
# importing LangChain modules
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import DatetimeOutputParser, CommaSeparatedListOutputParser


# Insert your key here
llm = OpenAI(openai_api_key = "{{api_key}}")

parser_dateTime = DatetimeOutputParser()
parser_List = CommaSeparatedListOutputParser()

# creating our prompt template
template = """Provide the response in format {format_instructions}
            to the user's question {question}"""

prompt_dateTime = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": parser_dateTime.get_format_instructions()},
)

prompt_List = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": parser_List.get_format_instructions()},
)

# formatting the output
print(llm.predict(text = prompt_dateTime.format(question="When was the first iPhone launched?")))
print(llm.predict(text = prompt_List.format(question="What are the four famous chocolate brands?")))

Pydantic (JSON) parser

In [ ]:
from typing import List
from langchain.llms import OpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator

# insert your key here
model = OpenAI(openai_api_key = "{{api_key}}")

# defining the author class for the model
class Author(BaseModel):
    number: int = Field(description="number of books written by the author")
    books: List[str] = Field(description="list of books they wrote")

user_query = "Generate the books written by Dan Brown."

# defining the output parser
output_parser = PydanticOutputParser(pydantic_object=Author)

# defining the prompt template with parser instructions
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

# defining the prompt
my_prompt = prompt.format_prompt(query=user_query)

# coverting the output to a string while generating the response
output = model(my_prompt.to_string())

# printing the result
print(output_parser.parse(output))

LLM Chain

In [ ]:
# importing the modules
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# defining the LLM model
llm = OpenAI(temperature=0.0, openai_api_key="{{api_key}}")

# creating the prompt template
prompt_template = PromptTemplate(
    input_variables=["book"],
    template="Name the author of the book {book}?",
)

# creating the chain
chain = LLMChain(llm=llm,
                prompt=prompt_template,
                verbose=True)

# calling the chain
print(chain.run("The Da Vinci Code"))

Simple sequential chain

In [ ]:
# importing the modules
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain

# defining the LLM model for the first chain
llm = OpenAI(temperature=0.0, openai_api_key="{{api_key}}")

# creating the prompt template and the first chain
prompt_1 = PromptTemplate(
    input_variables=["book"],
    template="Name the author who wrote the book {book}?"
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

# creating the prompt template and the second chain
prompt_2 = PromptTemplate(
    input_variables=["author_name"],
    template="Write a 50-word biography for the following author:{author_name}"
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

# combining the chains into a simple sequential chain
simple_sequential_chain = SimpleSequentialChain(chains=[chain_1, chain_2],verbose=True)

# running the simple sequential chain
simple_sequential_chain.run("The Da Vinci Code")

Multiple input sequential chain

In [ ]:
# importing the modules
from langchain.llms import OpenAI
from langchain.chains import LLMChain

from langchain.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain

# defining the LLM model
llm = OpenAI(temperature=0.0, openai_api_key="{{api_key}}")

# defining the original input
biography = "He is an American author of thriller fiction, best known for his Robert Langdon series. \
          He has sold over 200 million copies of his books, which have been translated into 56 \
          languages. His other works include Angels & Demons, The Lost Symbol, Inferno, and Origin. \
          He is a New York Times best-selling author and has been awarded numerous awards for his \
          writing."

# creating the prompt template for the first chain
prompt_1 = ChatPromptTemplate.from_template(
    "Summarize this biography in one sentence:"
    "\n\n{biography}"
)

# we input the original biography, and the output here is the one-line biography
chain_1 = LLMChain(llm=llm, prompt=prompt_1, output_key="one_line_biography")

# creating the prompt template for the second chain
prompt_2 = ChatPromptTemplate.from_template(
    "Can you tell the author's name in this biography:"
    "\n\n{one_line_biography}"
)

# we input the one-line biography, and the output here is the author's name
chain_2 = LLMChain(llm=llm, prompt=prompt_2, output_key="author_name")

# creating the prompt template for the third chain
prompt_3 = ChatPromptTemplate.from_template(
    "Tell the name of the highest selling book of this author: "
    "\n\n{author_name}"
)

# we input the author's name, and the output here is the highest-selling book
chain_3 = LLMChain(llm=llm, prompt=prompt_3, output_key="book")

# creating the prompt template for the fourth chain
prompt_4 = ChatPromptTemplate.from_template(
   "Write a follow-up response to the following "
    "summary of the highest-selling book of the author:"
    "\n\nAuthor: {author_name}\n\nBook: {book}"
)

# we input the author's name and the highest-selling book, and the output is the book's summary
chain_4 = LLMChain(llm=llm, prompt=prompt_4, output_key="summary")

# combining the chains
final_chain = SequentialChain(
    chains=[chain_1, chain_2, chain_3, chain_4],
    input_variables=["biography"],
    output_variables=["one_line_biography", "author_name","summary"],
    verbose=True
)

# calling the final chain
print(final_chain(biography))

ChatBot using router chain

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

# Defining prompt templates for the destination chains
shipping_template = """You are the shipping manager of a company. \
As a shipping customer service agent, respond to a customer inquiry about the current status \
and estimated delivery time of their package. Include details about the \
shipping route and any potential delays, providing a comprehensive and reassuring response. \

Here is a question:
{input}"""

billing_template = """You are the billing manager of a company. \
Address a customer's inquiry regarding an unexpected charge on their account.\
Explain the nature of the charge and any relevant billing policies, and promptly resolve \
the concern to ensure customer satisfaction. Additionally, offer guidance on how the \
customer can monitor and manage their billing preferences moving forward.

Here is a question:
{input}"""

technical_template = """You are very good at understanding the technology of your company's product. \
Assist a customer experiencing issues with a software application. \
Walk them through troubleshooting steps, provide clear instructions \
on potential solutions, and ensure the customer feels supported\
throughout the process. Additionally, offer guidance on preventive \
measures to minimize future technical issues and optimize their \
experience with the software.

Here is a question:
{input}"""

# storing the prompt templates in prompt_infos
prompt_infos = [
    {
        "name": "Shipping",
        "description": "Good for answering questions about shipping issues of a product",
        "prompt_template": shipping_template
    },
    {
        "name": "Billing",
        "description": "Good for answering questions regarding billing issues of a product",
        "prompt_template": billing_template
    },
    {
        "name": "Technical",
        "description": "Good for answering questions regarding technical issues of a product",
        "prompt_template": technical_template
    }
]

# creating the destination chains
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

# creating the default chain
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

# defining the router template
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

# creating the router chain
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# combining the chains
final_chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

# running the combined chain
print(final_chain.run("The package was supposed to arrive last week and I haven't received it yet. I want to cancel my order NOW!"))

Conversation buffer

In [ ]:
# importing LangChain modules
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain


# Insert your key here
llm = OpenAI(temperature=0.0,
            openai_api_key = "{{api_key}}")

memory = ConversationBufferMemory()
memory.save_context({"input": "Alex is a 9-year old boy."},
                    {"output": "Hello Alex! How can I assist you today?"})
memory.save_context({"input": "Alex likes to play football"},
                    {"output": "That's great to hear! "})

conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

print(conversation.predict(input="How old is Alex?"))

Conversation window

In [ ]:
# importing LangChain modules
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain


# Insert your key here
llm = OpenAI(temperature=0.0,
            openai_api_key = "{{api_key}}")

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({"input": "Alex is a 9-year old boy."},
                    {"output": "Hello Alex! How can I assist you today?"})
memory.save_context({"input": "Alex likes to play football"},
                    {"output": "That's great to hear! "})

conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

print(conversation.predict(input="How old is Alex?"))

Conversation summary

In [ ]:
# importing LangChain modules
from langchain.llms import OpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationChain


# Insert your key here
llm = OpenAI(temperature=0.0,
            openai_api_key = "{{api_key}}")

memory = ConversationSummaryMemory(llm=llm)
memory.save_context({"input": "Alex is a 9-year old boy."},
                    {"output": "Hello Alex! How can I assist you today?"})
memory.save_context({"input": "Alex likes to play football"},
                    {"output": "That's great to hear! "})

conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

print(conversation.predict(input="How old is Alex?"))

A chatbot that identifies the language of the user queries and answers them accordingly using the router chain.

In [ ]:
# importing the modules
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

# defining the LLM model
llm = OpenAI(temperature=0.0, openai_api_key="{{api_key}}")

# Defining prompt templates for the destination chains
french_template = """You are proficient in the French language and are very knowledgeable. \
 Translate the customer query to English and then respond to the user in French. \
 Be polite and have a friendly tone. \

Here is a question:
{input}"""

spanish_template = """You are proficient in the Spanish language and are very knowledgeable. \
 Translate the user query to English and then respond to the user in Spanish. \
 Be polite and have a friendly tone. \

Here is a question:
{input}"""

dutch_template = """You are proficient in the Dutch language and are very knowledgeable. \
 Translate the user query to English and then respond to the user in Dutch. \
 Be polite and have a friendly tone. \

Here is a question:
{input}"""

italian_template = """You are proficient in the Italian language and are very knowledgeable. \
 Translate the user query to English and then respond to the user in Italian. \
 Be polite and have a friendly tone. \

Here is a question:
{input}"""

# storing the prompt templates in prompt_infos
prompt_infos = [
    {
        "name": "French",
        "description": "Good for answering questions in French",
        "prompt_template": french_template
    },
    {
        "name": "Spanish",
        "description": "Good for answering questions in Spanish",
        "prompt_template": spanish_template
    },
    {
        "name": "Dutch",
        "description": "Good for answering questions in Dutch",
        "prompt_template": dutch_template
    },
    {
        "name": "Italian",
        "description": "Good for answering questions in Italian",
        "prompt_template": italian_template
    }
]

# creating the destination chains
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

# creating the default chain
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

# defining the router template
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

# defining the router prompt template
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

# creating the router chain
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# combining the chains
final_chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

# running the combined chain
print(final_chain.run("Cos'è un atomo?"))

Getting information from an external API - React agent

In [ ]:
import os
# importing LangChain modules
from langchain.llms import OpenAI
from langchain.agents import AgentType, initialize_agent, load_tools

os.environ["SERPAPI_API_KEY"] = "{{SERPAPI_API_KEY}}"

# Insert your key here
llm = OpenAI(temperature=0.0,
            openai_api_key = "{{api_key}}")

# loading tools
tools = load_tools(["serpapi",
                    "llm-math"],
                    llm=llm)

agent = initialize_agent(tools,
                        llm,
                        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                        verbose=True)

# user's query
print(agent.run("What is the current population of the world, and calculate the percentage change compared to the population five years ago"))

Empowering ChatGPT with current knowledge - Conversational react

In [ ]:
import os
# importing LangChain modules
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentType, initialize_agent, load_tools

os.environ["SERPAPI_API_KEY"] = "{{SERPAPI_API_KEY}}"
# Insert your key here
llm = OpenAI(temperature=0.0,
            openai_api_key = "{{api_key}}")

memory = ConversationBufferMemory(memory_key="chat_history")

# loading tools
tools = load_tools(["serpapi"],
                    llm=llm)

agent = initialize_agent(tools,
                        llm,
                        agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
                        verbose=True,
                        memory=memory)

agent.run("Hi, my name is Alex, and I live in the New York City.")
agent.run("My favorite game is basketball.")
print(agent.run("Give me the list of stadiums to watch a basketball game in my city today. Also give the teams that are playing."))

Document loaders

In [ ]:
# Loading a text file
from langchain.document_loaders import TextLoader

loader = TextLoader("inputFile.txt")
loader.load()

# Loading a csv file
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='inputRecord.csv')
data = loader.load()

# Loading a pdf file
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("inputDocument.pdf")
pages = loader.load_and_split()

Document transformers

In [ ]:
# Text splitters
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # The chunk_size and chunk_overlap can be modified according to the requirements
    length_function = len,
    chunk_size = 200,
    chunk_overlap  = 10,
    add_start_index = True,
)

texts = text_splitter.create_documents([input_document])

Text embedding

In [ ]:
# embedding doc
embeddings = embeddings_model.embed_documents(
 [
     "This is the first text",
     "This is the second text",
     "This is the third text"
 ]
)

# embedding a query
embedded_query = embeddings_model.embed_query("WRITE_YOUR_QUERY_HERE")

Computing a simple similarity index for searching data

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Chroma module is an open-source vector store for building AI applications.
from langchain.vectorstores import Chroma

# Load the document
input_document = TextLoader('my_document.txt').load()

# transform the document
text_splitter = RecursiveCharacterTextSplitter(
    # The chunk_size and chunk_overlap can be modified according to the requirements
    length_function = len,
    chunk_size = 200,
    chunk_overlap  = 10,
    add_start_index = True,
)

documents = text_splitter.create_documents([input_document])

# embed the chunks
db = Chroma.from_documents(documents, OpenAIEmbeddings())

# user query
query = "WRITE_YOUR_QUERY_HERE"

# computing the search using the similarity_search() method
docs = db.similarity_search(query)

Using vector stores to compute the similarity

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

# Load the document
input_document = TextLoader('my_document.txt').load()

# transform the document
text_splitter = RecursiveCharacterTextSplitter(
    # The chunk_size and chunk_overlap can be modified according to the requirements
    length_function = len,
    chunk_size = 200,
    chunk_overlap  = 10,
    add_start_index = True,
)

documents = text_splitter.create_documents([input_document])

# embed the chunks
db = Chroma.from_documents(documents, OpenAIEmbeddings())

# user query
query = "WRITE_YOUR_QUERY_HERE"

# embedding the query
embedding_vector = OpenAIEmbeddings().embed_query(query)

# computing the search using the search_by_vector() method
docs = db.similarity_search_by_vector(embedding_vector)

Retrievers

In [ ]:
from langchain.vectorstores import Chroma

db = Chroma.from_texts(texts, embeddings)
retriever = db.as_retriever()

# invoking the retriever
retrieved_docs = retriever.invoke(
    # write your query here
)

Using RAG to implement question answering
We’ll build a chatbot that answers the user’s queries using the PDF and a memory of the conversation history.

In [ ]:
from langchain.document_loaders import PyPDFLoader

# loading the document
loader = PyPDFLoader("./3DPrinter_Manual.pdf")
mypdf = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Defining the splitter
document_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap = 70
)

# splitting the document
docs = document_splitter.split_documents(mypdf)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# embedding the chunks to vector stores
embeddings = OpenAIEmbeddings(openai_api_key="{{api_key}}")
persist_directory = 'db'

my_database = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

# defining the conversational memory
retaining_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

In [ ]:
from langchain.chains import ConversationalRetrievalChain

# defining the retriever
question_answering = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=my_database.as_retriever(),
    memory=retaining_memory
)

In [ ]:
# defining the loop for a conversation with the AI
while True:
    question = input("Enter your query: ")
    if question == 'exit':
	    break
    # getting the response
    result = question_answering({"question": "Answer only in the context of the document provided." + question})
    print(result['answer'])

Agents for Question Answering with RAG

In [ ]:
# importing the modules
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import Chroma
from langchain.agents import Tool
from langchain.agents import initialize_agent

# defining the model
llm = ChatOpenAI(
    openai_api_key="{{api_key}}",
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

# loading the document
loader = PyPDFLoader("./3DPrinter_Manual.pdf")
mypdf = loader.load()

# Defining the splitter
document_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap = 70
)

# splitting the document
docs = document_splitter.split_documents(mypdf)

# embedding the chunks to vector stores
embeddings = OpenAIEmbeddings(openai_api_key="{{api_key}}")
persist_directory = 'db'

my_database = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory=persist_directory
)

# defining the conversational memory
retaining_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

# defining the retriever
question_answering = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=my_database.as_retriever(),
    memory=retaining_memory
)

# defining the tool for the agent
tools = [
    Tool(
        name='Knowledge Base',
        func=question_answering.run,
        description=(
            'use this tool when answering questions related to the 3D printer'
        )
    )
]

# initializing the agent
agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=retaining_memory
)

# calling the agent
while True:
    question = input("Enter your query: ")
    if question == 'exit':
	    break
    print(agent(question))
